In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tqdm import tqdm_notebook as tqdm

In [3]:
import pandas as pd
import numpy as np
import datetime
from pathlib import Path
from tadpole.models import SimpleSVM
from tadpole.utils import test_train_split

tadpoleD1D2File = Path("/Users/jvdzwaan/data/TADPOLE/data/TADPOLE_D1_D2.csv")

df = pd.read_csv(tadpoleD1D2File)

# remove all rows without
df = df[df["Ventricles"].notnull()]
df = df[df["DXCHANGE"].notnull()]
df = df[df["ADAS13"].notnull()]

train_df, test_df, ground_truth_df = test_train_split(df, 0.1)

model = SimpleSVM()
model.train(train_df)

# predict for a single patient
first_patient_id = test_df['RID'][0]
prediction = model.predict_all_months(test_df[test_df['RID'] == first_patient_id])
truth = ground_truth_df[ground_truth_df['RID'] == first_patient_id]

# evaluate (prediction, truth)

/Users/jvdzwaan/miniconda3/envs/tadpole/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,71

* CognitiveAssessmentDate = ExamDate (source: https://github.com/tadpole-share/TADPOLE-eval/blob/c8d4e241bc143b858d9b8237aab92417d3e871e2/evaluation/makeLeaderboardDataset.py#L159)
* ScanDate = ExamDate (source: https://github.com/tadpole-share/TADPOLE-eval/blob/c8d4e241bc143b858d9b8237aab92417d3e871e2/evaluation/makeLeaderboardDataset.py#L162)
* Diagnosis = DXCHANGE with mapping (source: https://github.com/tadpole-share/TADPOLE-eval/blob/c8d4e241bc143b858d9b8237aab92417d3e871e2/evaluation/makeLeaderboardDataset.py#L160 and line 169-170)
* RID = Patient id (what is it used for? Why must it be set?)

In [4]:
from tadpole.transformations import get_diagnosis

# The Forecast date has to be a string
p = prediction.copy()
p['Forecast Date'] = p['Forecast Date'].astype(str)
print(p.dtypes)

# Add expected columns
truth['CognitiveAssessmentDate'] = truth['EXAMDATE']
truth['ScanDate'] = truth['EXAMDATE']

truth['Diagnosis'] = get_diagnosis(truth)

AD relative probability        float64
ADAS13                         float64
ADAS13 50% CI lower            float64
ADAS13 50% CI upper            float64
CN relative probability        float64
Diagnosis                      float64
Forecast Date                   object
MCI relative probability       float64
RID                              int64
Ventricles_ICV                 float64
Ventricles_ICV 50% CI lower    float64
Ventricles_ICV 50% CI upper    float64
dtype: object


/Users/jvdzwaan/miniconda3/envs/tadpole/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/jvdzwaan/miniconda3/envs/tadpole/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/jvdzwaan/miniconda3/envs/tadpole/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [5]:
def outcomes_equal(result1, result2):
    """Method that checks whether two evaluation outcomes are equal.
    """
    for i1, i2 in zip(result1, result2):
        if isinstance(i1, pd.Series):
            r = i1.equals(i2)
        else:
            r = i1 == i1 or np.isnan(i1) and np.isnan(i2)

        if not r:
            return False
    return True

In [6]:
%%time
from evaluation import evalOneSubmissionExtended, simpleEvalOneSubmissionExtended

res1 = evalOneSubmissionExtended.evalOneSub(truth.copy(), p.copy())
res2 = simpleEvalOneSubmissionExtended.evalOneSub(truth.copy(), p.copy())
                           
assert outcomes_equal(res1, res2)

a_value 0 0 0
a_value 0 0 0
a_value 0 1 0
a_value 1 0 1
a_value 0 1 0
a_value 1 0 1
[[0 0 0]
 [0 0 0]
 [0 0 1]]
Number of subjects: 1
1 (2, [0.30000000000000004, 0.30000000000000004, 2.4000000000000004])
a_value 0 0 0
a_value 0 0 0
a_value 0 1 0
a_value 1 0 1
a_value 0 1 0
a_value 1 0 1
CPU times: user 44.4 ms, sys: 3.75 ms, total: 48.2 ms
Wall time: 47 ms


In [8]:
%%time
from tadpole.transformations import get_diagnosis
from evaluation import evalOneSubmissionExtended, simpleEvalOneSubmissionExtended

result = model.predict_all_months(test_df.copy())

# Add expected columns
#ground_truth_df['CognitiveAssessmentDate'] = ground_truth_df['EXAMDATE']
#ground_truth_df['ScanDate'] = ground_truth_df['EXAMDATE']
#ground_truth_df['Diagnosis'] = get_diagnosis(truth)

#res1 = evalOneSubmissionExtended.evalOneSub(ground_truth_df.copy(), result.copy())
#res2 = simpleEvalOneSubmissionExtended.evalOneSub(ground_truth_df.copy(), result.copy())
                           
#assert outcomes_equal(res1, res2)

/Users/jvdzwaan/code/tadpole/jupyter/tadpole/utils.py:43: RankWarning: Polyfit may be poorly conditioned
  lm = np.polyfit(x,y,1)
/Users/jvdzwaan/code/tadpole/jupyter/tadpole/utils.py:43: RankWarning: Polyfit may be poorly conditioned
  lm = np.polyfit(x,y,1)
/Users/jvdzwaan/code/tadpole/jupyter/tadpole/utils.py:43: RankWarning: Polyfit may be poorly conditioned
  lm = np.polyfit(x,y,1)
/Users/jvdzwaan/code/tadpole/jupyter/tadpole/utils.py:43: RankWarning: Polyfit may be poorly conditioned
  lm = np.polyfit(x,y,1)
/Users/jvdzwaan/code/tadpole/jupyter/tadpole/utils.py:43: RankWarning: Polyfit may be poorly conditioned
  lm = np.polyfit(x,y,1)
/Users/jvdzwaan/code/tadpole/jupyter/tadpole/utils.py:43: RankWarning: Polyfit may be poorly conditioned
  lm = np.polyfit(x,y,1)
/Users/jvdzwaan/code/tadpole/jupyter/tadpole/utils.py:43: RankWarning: Polyfit may be poorly conditioned
  lm = np.polyfit(x,y,1)
/Users/jvdzwaan/code/tadpole/jupyter/tadpole/utils.py:43: RankWarning: Polyfit may be poor

IndexError: single positional indexer is out-of-bounds

In [ ]:
# predict for all patients
for patient_id, patient_data in test_df.groupby('RID'):
    print(patient_id)
    #print(patient_data)
    pred = model.predict(patient_data, datetime.datetime.now())
    gt = ground_truth_df(ground_truth_df['RID'] == patient_id)

In [6]:
res1

(0.0,
 0.6666666666666666,
 0.6225826460276949,
 97419.95826182756,
 0.6225826460276949,
 97419.95826182756,
 0.5,
 0.5,
 array([37.04741735]),
 0    37.67
 Name: ADAS13, dtype: float64)

In [7]:
res2

(0.0,
 0.6666666666666666,
 0.6225826460276949,
 97419.95826182756,
 0.6225826460276949,
 97419.95826182756,
 0.5,
 0.5,
 array([37.04741735]),
 0    37.67
 Name: ADAS13, dtype: float64)

In [ ]:
for i, (i1, i2) in enumerate(zip(res1, res2)):
    print(i, i1==i2, type(i1))

In [ ]:
import numpy as np

np.nan == np.nan

The evaluation code expects as input:

* `d4Df`: Pandas data frame containing the D4 dataset
  * 
  * Required fields in the DataFrame:
    * 
* `forecastDf`: Pandas data frame containing user forecasts for D2 subjects.
  * Predictions
  * Required fields in the DataFrame:
    * Diagnosis
    * ADAS13
    * Ventricles_ICV
    * Forecast Date (string in the format `%Y-%m`)
    * RID: Patient ID
    * CN relative probability
    * MCI relative probability
    * AD relative probability
    * ADAS13 50% CI lower
    * ADAS13 50% CI upper
    * Ventricles_ICV 50% CI lower
    * Ventricles_ICV 50% CI upper

In [ ]:
import datetime

a = datetime.datetime.now()
b = a.strftime('%Y-%m')
type(b)

In [ ]:
datetime.datetime.strptime(b, '%Y-%m')

In [ ]:
def average_time_between_visits(df):
    df1 = df[['RID', 'EXAMDATE']].copy()
    df1 = df1.sort_values(by=['RID', 'EXAMDATE'])
    df1['EXAMDATE'] = df1['EXAMDATE'].map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    # Get future value from each row's next row
    df1["Future_EXAMDATE"] = df1['EXAMDATE'].shift(-1)
    # Drop each last row per patient
    df1 = df1.drop(df1.groupby('RID').tail(1).index.values)
    return (df1['Future_EXAMDATE']-df1['EXAMDATE']).mean()


In [ ]:
average_time_between_visits(df)

In [ ]:
truth